# SEDI Index

In [ ]:
# general use packages
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np

# packages for altering time to match up!
import sys
import cftime

# climpred packages
import climpred
from climpred import HindcastEnsemble
from climpred.tutorial import load_dataset
from climpred.stats import rm_poly

# SMYLE Utility functions
from SMYLEutils import io_utils as io
from SMYLEutils import calendar_utils as cal
from SMYLEutils import stat_utils as stat

In [ ]:
import random

In [ ]:
import sklearn.metrics as sk

In [ ]:
var = 'TEMP'
var2 = 'TEMP'
depth = 'surface'

In [ ]:
smyle02 = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.02.binary.Rolling.full.2.nc')['binary']
smyle02_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.02.time.nc')#.sel(L=slice(1,12))

smyle05 = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.05.binary.Rolling.full.2.nc')['binary']
smyle05_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.05.time.nc')#.sel(L=slice(1,12))

smyle08 = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.08.binary.Rolling.full.2.nc')['binary']
smyle08_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.08.time.nc')#.sel(L=slice(1,12))

smyle11 = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.11.binary.Rolling.full.2.nc')['binary']
smyle11_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.11.time.nc')#.sel(L=slice(1,12))

In [ ]:
if var == 'combined_oa':
    smyle02 = ~np.isnan(smyle02)
    smyle05 = ~np.isnan(smyle05)
    smyle08 = ~np.isnan(smyle08)
    smyle11 = ~np.isnan(smyle11)

In [ ]:
smyle02 = smyle02.expand_dims("init"); smyle02_time  = smyle02_time.expand_dims("init")
smyle05 = smyle05.expand_dims("init"); smyle05_time  = smyle05_time.expand_dims("init")
smyle08 = smyle08.expand_dims("init"); smyle08_time  = smyle08_time.expand_dims("init")
smyle11 = smyle11.expand_dims("init"); smyle11_time  = smyle11_time.expand_dims("init")

In [ ]:
smyle02['init'] = ['02']; smyle02_time['init'] = ['02']
smyle05['init'] = ['05']; smyle05_time['init'] = ['05']
smyle08['init'] = ['08']; smyle08_time['init'] = ['08']
smyle11['init'] = ['11']; smyle11_time['init'] = ['11']

In [ ]:
smyle = xr.concat([smyle02,smyle05,smyle08,smyle11],'init')
smyle_time = xr.concat([smyle02_time,smyle05_time,smyle08_time,smyle11_time],'init')

In [ ]:
# var = 'omega_arag'
# obs = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/' + var + '.mri.rolling.thold.Rolling.nc')
obs = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/' + var + '.obs.rolling.thold.Rolling2.nc')
# obs = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/' + var2 + '.fosi.rolling.thold.Rolling.nc')
# var = 'omega_arag'

## mask land as NaNs

In [ ]:
ds = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/OceanSODA-ETHZ_GRaCER_v2021a_1982-2020.nc')['temperature'].isel(time=0).drop('time')

In [ ]:
ds = ds.where(ds.lat < 65)

In [ ]:
np.isnan(ds).plot()

In [ ]:
obs = obs.where(np.isnan(ds) == 0, np.NaN)

In [ ]:
obs.threshold.sel(time='1997-05').plot()

In [ ]:
smyle = smyle.where(np.isnan(ds) == 0, np.NaN)

In [ ]:
smyle.sel(Y=1997,M=1,L=4,init='02').plot()

## Make a contingency table

| Value      | Description |
| ----------- | ----------- |
| 1      | False negative       |
| 2      | True negative       |
| 3      | False positive       |
| 4      | True positive       |


In [ ]:
def sedi_index(data):
    hit_rate = (~np.isnan(data.where(data == 4))).sum(dim=('member','time'),skipna=True) / ((~np.isnan(data.where(data == 4))).sum(dim=('member','time'),skipna=True) + (~np.isnan(data.where(data == 1))).sum(dim=('member','time'),skipna=True))
    hit_rate = hit_rate
    false_rate = (~np.isnan(data.where(data == 3))).sum(dim=('member','time'),skipna=True) / ((~np.isnan(data.where(data == 3))).sum(dim=('member','time'),skipna=True) + (~np.isnan(data.where(data == 2))).sum(dim=('member','time'),skipna=True))
    false_rate = false_rate
    num = np.log10(false_rate) - np.log10(hit_rate) - (np.log10(1- false_rate)) + np.log10(1 - hit_rate)
    denom = np.log10(false_rate) + np.log10(hit_rate) + (np.log10(1- false_rate)) + np.log10(1 - hit_rate)
    sedi = num / denom
    
    return sedi

In [ ]:
def skill(mod_da,mod_time,obs_da):
    # results = []
    sedi_results = []
    
    for i in range(len(mod_da.L)):
        # select a given member of the ensemble
        mod_da_L = mod_da.isel(L=i)
        results = []
        obs_range = []
        # put everything into a contingency table (key is listed above)
        for h in (mod_da.init):
            mod_da_L_I = mod_da_L.sel(init=h)
            for j in range(6): # range(len(mod_da.M)):
                ens = mod_da_L_I.isel(M=j).rename({'Y':'time'})
            
                ens_time_year = mod_time.sel(init=h).isel(L=i).time.dt.year.data
                ens_time_month = mod_time.sel(init=h).isel(L=i).time.dt.month.data[0]
                ens_ts = ens.assign_coords(time=("time",ens_time_year))
                obs_ts = obs_da.where(obs_da.time.dt.month == ens_time_month,drop=True)
            
                obs_ts['time'] = obs_ts['time'].dt.year
            
                ens_ts, obs_ts = xr.align(ens_ts, obs_ts)
                
                ## randomize the values here!
                ens_ts_use = random.choices(ens_ts,k=len(ens_ts))
                ens_ts_use = xr.concat(ens_ts_use,'time')
                ens_ts_use['time'] = ens_ts['time']
                
                pos = (ens_ts_use.where(ens_ts_use.astype(int) == 1) + obs_ts + 2).rename('skill')
                neg = (ens_ts_use.where(ens_ts_use.astype(int) == 0) - obs_ts + 2).rename('skill')
                contingency = xr.merge([pos,neg])
                contingency = contingency.expand_dims('member')
                results.append(contingency)
                
                # isolate the month from observations 
                obs_ts.expand_dims('month')
                obs_ts['month'] = ens_time_month
                obs_range.append(obs_ts)                
        
        results_ds = xr.concat(results,'member')
        
        # run the SEDI score function (above), as in Jacox 2022
        sedi = sedi_index(results_ds.skill)
        sedi_results.append(sedi)            
        
    ds = xr.concat(sedi_results,'L'); 
    
    return ds
    # return ens_ts, ens_ts_use, obs_ts

In [ ]:
%%time
# contingency, results = skill(smyle,smyle_time,obs.threshold)
sedi_score = skill(smyle,smyle_time,obs.threshold)

In [ ]:
%%time
sedi_bootstrap = []
# for i in range(1000):
for i in range(1000):
    print(i)
    sedi_score = skill(smyle,smyle_time,obs.threshold)
    sedi_score.expand_dims('bootstrap')
    sedi_score['bootstrap'] = i
    sedi_bootstrap.append(sedi_score)

In [ ]:
# 100 times, 10 members
tmp = xr.concat(sedi_bootstrap,'bootstrap')
tmp.quantile(.995,dim='bootstrap').plot(vmin=-1,vmax=1,cmap='coolwarm')

In [ ]:
tmp.quantile(.975,dim='bootstrap').sel(lat=10.5,lon=-150.5)#.mean()

In [ ]:
tmp.to_netcdf('./' + var + '.signficance.lead.nc')

In [ ]:
tmp.quantile(.995,dim='bootstrap').sel(lat=10.5,lon=-150.5)#.mean()